<a href="https://colab.research.google.com/github/StevenHSKim/tensorflow_study/blob/main/pj4_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%A6%9D%EA%B0%95(Augmentation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#이미지 증강 Image Augmentation

이미지 학습을 잘하고 싶으면 이미지의 양을 늘리십시오. 그러면 언제나 더 좋은 결과를 가져옵니다.

하지만 이미지 증강 Augmentation 이라는 테크닉도 있습니다.

이미지를 살짝살짝씩 돌리고 뒤집고 변형해서 딥러닝을 돌리는겁니다.

그럼 모델은 각각 다른 이미지라고 인식하니까요.

그럼 조금 더 overfitting이 완화되고, accuracy도 높아집니다.

텐서플로우에선 어떻게 해야하는지 알아봅시다.  





텐서플로우의 모든 전처리 레이어는 여기 있습니다.

https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing

###Kaggle에서 데이터셋 사용하는 방법
1. os로 다운로드 받은 후, !kaggle로 코랩 content에 저장
2. !unzip으로 압축 풀기
3. 이미지 전처리 작업에서 cat,dog 폴더 직접 만들어주기(/dataset/ 폴더 안에 /cat/과 /dog/ 만들기)

In [ ]:
# 코랩에서 kaggle dataset 다운로드
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats

100% 811M/812M [00:21<00:00, 42.8MB/s]
100% 812M/812M [00:21<00:00, 39.8MB/s]


In [ ]:
# 코랩에서 압축 풀기
!unzip -q dogs-vs-cats.zip -d .
!unzip -q train.zip -d .

In [ ]:
# 이미지 파일 개수 세기
import os
print(len(os.listdir('/content/train/')))

25000


#이미지 전처리

In [ ]:
import tensorflow as tf
import shutil

# tf.keras.preprocessing.image_dataset_from_directory() 이용하기 위한 사전 작업 - cat/dog 폴더 구분
# 먼저, content에 dataset 폴더 만들고 cat, dog 폴더 만들기
for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
  if 'dog' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)

In [ ]:
# tf.keras 이용해서 이미지를 숫자화
# train_ds에는 ( x(이미지를 행렬로 바꾼 데이터), y(개,고양이 정답 0 또는 1) )가 담김
# train_ds를 모델에 집어넣으면 학습 끝
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=32,

    # validation data도 준비
    subset='training',
    validation_split=0.2, # 20% 만큼 validation dataset으로 쪼개기
    seed=1234
)

# validation dataset 만들 때 이 형식 따라야 함.
# 위 train_ds은 'training'으로 이름짓고, 아래 val_ds는 'validation'으로 이름 짓기
# train_ds는 전체 데이터 중 80%, val_ds는 전체 데이터 중 20%
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=32,

    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)
# print 해보면, 위에서 언급한 대로
# ((shape=(None, 64, 64, 3), (shape=(None,)) 모양으로 나오는 것을 알 수 있음.


#------------------성능 높히는 코드(보통 사용함)------------------
# 이미지 데이터는 현재 텐서 형태로 0~255의 값을 가지고 있음. 이를 0~1로 압축하기
def preprocessing_function(i, 정답):
  i = tf.cast(i/255.0, tf.float32) # 텐서를 다루기 때문에 i = i/255.0 불가 -> tf.cast() 사용
  return i, 정답

# map(func): 데이터에 전부 func를 적용해라
train_ds = train_ds.map(preprocessing_function)
val_ds = val_ds.map(preprocessing_function)
#-----------------------------------------------------------

# 0~1로 압축한 모습 확인
# for i, 정답 in train_ds.take(1): # take(1): 1개만 뽑아서 확인
#   print(i)
#   print(정답)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


#모델 만들기 & 이미지 증강하기

## 방법1: 모델에 이미지 넣기 전에 전처리 layer을 만듦 (최근 나온 쉬운 방식의 이미지 증강)

On-the-fly(Conv모델에 이미지 집어넣기 전에) 이미지에 변형을 줌.
예를 들어 epoch=10이라고 하면,

*   이미지1.jpg가 들어갈 때마다 같은 이미지로 들어가는 것이 아니라
*   epoch의 반복마다 이미지1.jpg의 다른 버전(뒤집기, 돌리기, 확대하기)이 들어감



## 이미지 증강 시 기존과 차이점:
* 기존 방식에서는 train_acc는 높지만, val_acc는 상대적으로 낮은 overfitting 문제가 있지만
* 이미지 증강 시, train_acc과 val_acc이 비슷해져 overfitting 문제를 해결한다

In [ ]:
model = tf.keras.Sequential([
    #------------------Preprocessing method로 데이터 증강------------------
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3)), # 임의로 뒤집기 # input_shape는 항상 처음에 넣기
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),      # 임의로 돌리기
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),          # 임의로 확대하기
    #-------------------------------------------------------------------

    # Convolution 3번 반복 해보기
    tf.keras.layers.Conv2D(32, (3,3) ,padding="same", activation='relu'), # RGB 채널: 3
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3) ,padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    # dropout: overfitting 방지 위해 노드가 많아보이면 줄임 (여기선 20%)
    tf.keras.layers.Dropout(0.2), # 위치는 아무데나. 보통 Conv에선 Pooling 끝나고.

    tf.keras.layers.Conv2D(128, (3,3) ,padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"), # binary_crossentropy 문제는 sigmoid로 - (분류 문제) 0~1 사이의 확률 제시
])

# 모델 Summary
model.summary()

# 모델 컴파일 및 실행
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, validation_data=(val_ds), epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 64, 64, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 64, 64, 3)         0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1

##방법 2: tensorflow 전통 방식 -> 이미지 전처리(rescaling) 함수를 사용

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

생성기 = ImageDataGenerator(
    rescale=1./255,         # 스케일링
    rotation_range=20,      # 회전
    zoom_range=0.15,        # 확대
    width_shift_range=0.2,  # 이동
    height_shift_range=0.2,
    shear_range=0.15,       # 굴절
    horizontal_flip=True,   # 가로반전
    fill_mode="nearest"
)

In [ ]:
# train data 전처리
# 아래 함수가 결국 "image_dataset_from_directory()"와 같은 것
트레이닝용 = 생성기.flow_from_directory(
    '/content/dataset',
    class_mode='binary', # 두 개면 binary, 이상이면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
    target_size=(64,64)
)

Found 25000 images belonging to 3 classes.


valdation data 전처리하기에 앞서, val_dataset/cat, val_dataset/dog 폴더 만들기

In [ ]:
# validation data 전처리 -> 절대 빼먹지 말기
# 이때, validation용 이미지는 뒤틀 필요가 없음 -> 따라서 뒤틂 작업 제외한 생성기2 선언
생성기2 = ImageDataGenerator(rescale=1./255) # 증강 작업 모두 빼고 나누기 255만

검증용 = 생성기2.flow_from_directory(
    '/content/val_dataset',
    class_mode='binary', # 두 개면 binary, 이상이면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
)

Found 0 images belonging to 3 classes.


In [ ]:
# model.fit()에 넣으면 됨
model.fit(
    트레이닝용,
    validation_data=검증용,
    steps_per_epoch=20000 // 64,
    epochs=1
)

312/312 [==============================] - 61s 187ms/step - loss: -99401637888.0000 - accuracy: 0.4942


#참고
tf.keras.preprocessing.image.ImageDataGenerator 말고

직접 이미지 증강용 레이어를 쓰는 식으로 하는 방법도 있는데

https://www.tensorflow.org/tutorials/images/data_augmentation#resizing_and_rescaling

텐서플로우 추후 버전에서는 이렇게 하라고 권장합니다.